In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

# runs full Langevin dynamics for LJ7 in 2D
beta = 5 # temperature T = 0.2

def LJpot(x): # Lennard-Jones potential
    Na = np.size(x,axis = 1)
    r2 = np.zeros((Na,Na)) # matrix of distances squared
    for k in range(Na):
        r2[k,:] = (x[0,:]-x[0,k])**2 + (x[1,:]-x[1,k])**2
        r2[k,k] = 1
    er6 = np.divide(np.ones_like(r2),r2**3)
    L = (er6-1)*er6
    V = 2*np.sum(L) 
    return V

def LJgrad(x):
    Na = np.size(x,axis = 1)
    r2 = np.zeros((Na,Na)) # matrix of distances squared
    for k in range(Na):
        r2[k,:] = (x[0,:]-x[0,k])**2 + (x[1,:]-x[1,k])**2
        r2[k,k] = 1
    r6 = r2**3
    L = -6*np.divide((2*np.divide(np.ones_like(r2),r6)-1),(r2*r6))
    g = np.zeros_like(x)
    for k in range(Na):
        Lk = L[:,k]
        g[0,k] = np.sum((x[0,k] - x[0,:])*Lk)
        g[1,k] = np.sum((x[1,k] - x[1,:])*Lk)
    g = 4*g
    return g

xmin = -2.5
xmax = 2.5
ymin = -2.5
ymax = 2.5
bbox = np.array([[xmin,xmax],[ymin,ymax]]) 

x = np.arange(0,14,dtype = float)
print(x)
y = np.reshape(x,(2,7))
print(y)
print("LJpot = ",LJpot(y))
g = LJgrad(y)
print("LJgrad = ",g)
gcol = np.reshape(g,(14,1))
print(gcol)
gmat = np.reshape(g,(2,7))
print(gmat)

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13.]
[[ 0.  1.  2.  3.  4.  5.  6.]
 [ 7.  8.  9. 10. 11. 12. 13.]]
LJpot =  -2.6671701358167184
LJgrad =  [[-1.13747471e+00 -1.24693563e-02 -7.77182993e-04 -7.24247051e-17
   7.77182993e-04  1.24693563e-02  1.13747471e+00]
 [-1.13747471e+00 -1.24693563e-02 -7.77182993e-04 -7.24247051e-17
   7.77182993e-04  1.24693563e-02  1.13747471e+00]]
[[-1.13747471e+00]
 [-1.24693563e-02]
 [-7.77182993e-04]
 [-7.24247051e-17]
 [ 7.77182993e-04]
 [ 1.24693563e-02]
 [ 1.13747471e+00]
 [-1.13747471e+00]
 [-1.24693563e-02]
 [-7.77182993e-04]
 [-7.24247051e-17]
 [ 7.77182993e-04]
 [ 1.24693563e-02]
 [ 1.13747471e+00]]
[[-1.13747471e+00 -1.24693563e-02 -7.77182993e-04 -7.24247051e-17
   7.77182993e-04  1.24693563e-02  1.13747471e+00]
 [-1.13747471e+00 -1.24693563e-02 -7.77182993e-04 -7.24247051e-17
   7.77182993e-04  1.24693563e-02  1.13747471e+00]]


In [ ]:
# leapfrog stochastic integrator
def LeapFrogStep(x,xm1,v,vm05,Ffun,w,dt,gfac,bbox): 
    # 0.5*gfac = gamma*dt
    # xm1 = x(t-dt)
    # vm05 = v(t-0.5*dt)
    # Ffun = force function
    # w ~ N(0,2*gamma*T/dt)
    v05 = vm05 + ((1-gfac)*(x-xm1)/dt + dt*(Ffun(x) + w))/(1+gfac)
    x1 = x + dt*v05
    # check if the positions are still in the bounding box
    ind_reflect_xmin = np.argwhere(x1[0,:] < bbox[0,0])
    if not np.any(ind_reflect_xmin):
        x1,v05 = reflect(0,xmin,x,xm1,x,v,vm05,v)
    ind_reflect_xmax = np.argwhere(x1[0,:] < bbox[0,0])
    if not np.any(ind_reflect_xmin):
        x1,v05 = reflect(0,xmax,x,xm1,x,v,vm05,v)

    
    
    
    return x1,v05

    